# YANDEX ML

In [33]:
import pandas as pd
import pymorphy2
from  text_preprocess2 import TextPreprocessor
import pickle
from  profiler import Profiler
import sklearn.cross_validation
import numpy as np

In [34]:
df = pd.read_csv('train.tsv',
                         names=['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply',
                                'label', 'confidence'],
                        header=None, sep='\t')

In [37]:
idies = np.array(sorted(list(set(df['context_id']))))
n = len(idies)
kf = sklearn.cross_validation.KFold (n = n,
                   n_folds = 4,
                   shuffle = True,
                   random_state = 40)
a = np.array([0,1,2,3,4,5,6,7,8,9])
for train_index, test_index in kf:
    X_train = idies[train_index]
    y_test = idies[test_index]

[    22579918886     72855813972     76323501777 ... 281428817239034
 281442741987260 281445178379747]
